# Quickstart: Grid & Derivative Operators
Demonstrates pvtend's NH grid setup and finite-difference operators on synthetic fields.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pvtend
from pvtend.grid import NHGrid, default_nh_grid
from pvtend.derivatives import ddx, ddy
from pvtend.constants import R_EARTH

print(f"pvtend version: {pvtend.__version__}")

## 1. Create Northern Hemisphere Grid
The default grid is 1.5° resolution, 90°N to 0°N, -180° to 180°.

In [ ]:
grid = default_nh_grid()
print(f"Grid: {grid.nlat} lat × {grid.nlon} lon")
print(f"Spacing: {grid.dlat}° × {grid.dlon}°")
print(f"Lat range: {grid.lat.min():.1f}° to {grid.lat.max():.1f}°")
print(f"Lon range: {grid.lon.min():.1f}° to {grid.lon.max():.1f}°")

## 2. Synthetic Pressure Field
Create a smooth field resembling a mid-latitude Rossby wave pattern.

In [ ]:
lon2d, lat2d = np.meshgrid(grid.lon, grid.lat)
# Rossby wave-like pattern: wavenumber 5
k = 5
field = 500 * np.cos(np.deg2rad(lat2d - 45) * 3) * np.sin(np.deg2rad(lon2d) * k)

fig, ax = plt.subplots(figsize=(12, 4))
im = ax.contourf(grid.lon, grid.lat, field, levels=20, cmap="RdBu_r")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("Synthetic Rossby Wave Pattern")
plt.colorbar(im, ax=ax, label="Amplitude")
plt.tight_layout()
plt.show()

## 3. Compute Derivatives
Zonal derivative `ddx` uses periodic wrapping; meridional `ddy` uses one-sided at boundaries.

In [ ]:
# Zonal grid spacing in metres (varies with latitude)
cos_lat = np.cos(np.deg2rad(grid.lat))
dx_arr = np.deg2rad(grid.dlon) * R_EARTH * cos_lat  # shape (nlat,)

dfdx = ddx(field, dx_arr, periodic=True)
dfdy = ddy(field, np.deg2rad(grid.dlat) * R_EARTH)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
for ax, data, title in zip(axes, [dfdx, dfdy],
                            ["∂f/∂x (zonal)", "∂f/∂y (meridional)"]):
    vmax = np.nanmax(np.abs(data))
    im = ax.contourf(grid.lon, grid.lat, data, levels=20,
                      cmap="coolwarm", vmin=-vmax, vmax=vmax)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_title(title)
    plt.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()

The zonal derivative shows strong gradients at the wave crests/troughs, while the meridional derivative highlights the jet-stream-like north-south structure.